In [1]:
import os

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

import numpy as np
np.set_printoptions(precision=3, suppress=True)

--2022-05-23 06:11:31--  https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py [following]
--2022-05-23 06:11:32--  https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48687 (48K) [text/plain]
Saving to: ‘thinkdsp.py’

thinkdsp.py         100%[===================>]  47.55K  --.-KB/s    in 0.01s   

2022-05-23 06:11:32 (3.64 MB/s) - ‘thinkdsp.py’ saved [48687/48687]



# Упражнение 7.1

Была проверена работа примеров в блокноте chap07.ipynb

# Упражнение 7.2

В главе 7 показано, как выразить ДПФ и обратное ДПФ произведениями матриц. Время выполнения этих операций пропорционально $N^2$, где $N$ — длина массива сигнала. Во многих случаях этого достаточно, но есть алгоритм и побыстрее — быстрое преобразование Фурье (БПФ); время его работы пропорционально $N$log$N$.

Ключ к БПФ - это лемма Дэниелсона-Ланцоша (Danielson-Lanczos):

DFT($y$)[$n$] = DFT($e$)[$n$] + exp(—2$\pi$$in/N$) DFT($o$)[$n$],

где DFT($y$)[$n$] — это $n$-й элемент ДПФ от $y$, $e$ и $o$ — массивы сигнала,
содержащие соответственно четные и нечетные элементы $y$.

Эта лемма предлагает рекурсивный алгоритм для ДПФ:

1. Дан массив сигнала $y$. Разделим его на четные элементы $e$ и нечетные элементы $o$.

2. Вычислим DFT $e$ и $o$, делая рекурсивные вызовы.

3. Вычислим DFT($y$) для каждого значения $n$, используя лемму Дэниелсона-Ланцоша.

В простейшем случае эту рекурсию надо продолжать, пока длина $y$ не дойдет до 1. Тогда DFT($y$) = $y$. А если длина $y$ достаточно мала, можно вычислить его ДПФ перемножением матриц, используя заранее вычисленные матрицы.


Для быстрого преобразования Фурье может быть использована библиотечная функция np.fft.fft

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


In [3]:
PI2 = 2 * np.pi
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

Результаты для БПФ совпадают с результатами получеными при дискретном преобразовании Фурье.

In [4]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Предложенная версия БПФ предполагает разделение массива сигнала $y$ на четные элементы $e$ и нечетные элементы $o$. Полученные подмассивы передаются в библиотечную функцию np.fft.fft

In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [6]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Вызов библиотечной функции np.fft.fft может быть заменён на рекурсивный вызов разработанной функции. 

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [8]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16